In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Nov 22 18:19:32 2023

@author: aksha
"""

import csv

# Function to extract first three letters from odd rows and last three from even rows
def extract_letters(file_path):
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        odd_first_three = set()
        even_last_three = set()

        for i, row in enumerate(reader, start=1):
            fx_name = row['FX_Name']
            if i % 2 != 0:  # Odd rows
                if fx_name != 'USD':
                    odd_first_three.add(fx_name[:3])
            else:  # Even rows
                if fx_name != 'USD':
                    even_last_three.add(fx_name[-3:])
    
    return odd_first_three, even_last_three

file_path = 'C:/Users/aksha/Downloads/data_eng_fin/profit and loss/Trading_Schedule.csv'

# Extract letters from FX_Name column in odd and even rows excluding 'USD'
odd_first_three, even_last_three = extract_letters(file_path)

# Merge unique values from both sets into a single list
merged_unique_list = list(odd_first_three | even_last_three)

# Print the merged unique list
print("Merged unique list of first three and last three letters (excluding 'USD'):")
print(merged_unique_list)


from polygon import RESTClient
client = RESTClient("beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq");
currency_values = {}
# Iterate through each unique currency and fetch real-time conversion rates
for currency in merged_unique_list:
    if currency != 'USD':  # Skip USD as it's the base currency
        res = client.get_real_time_currency_conversion("USD", currency, precision=5)
        currency_values[currency] = res.converted

print("Currency Values Dictionary:")
print(currency_values)


import pandas as pd

# Function to calculate contract equivalents
def calculate_contracts(currency_pair, direction, units=100):
    """
    Calculate the number of contracts based on the currency pair and direction.

    Parameters:
    currency_pair (str): The currency pair.
    direction (int): The direction of the trade. 1 for long, -1 for short.
    units (int, optional): The number of units. Default is 100.

    Returns:
    int: The number of contracts.

    """
    if direction == 1:
        last_three_letters = currency_pair[-3:]
        if last_three_letters in currency_values:
            currency_value = currency_values[last_three_letters]
            contracts = 100 * round(currency_value, 2)
            print(last_three_letters,contracts)
            # with open('CURRENCY.txt', 'a') as f:
            #     f.write('%s\n' % last_three_letters)
            return round(contracts)
        
        else:
            return 100
    elif direction == -1:
        first_three_letters = currency_pair[:3]
        if first_three_letters in currency_values:
            currency_value = currency_values[first_three_letters]
            contracts = 100 * round(currency_value, 2)
            print(first_three_letters,contracts)
            return round(contracts)
        else:
            return 100

    return None

# Function to update Trading_Schedule.csv and save as a new file
def save_new_file_with_contracts(input_file_path, output_file_path):
    # Load the Trading_Schedule.csv file
    schedule_df = pd.read_csv(input_file_path)

    # Create a new column 'Contract_Equivalents' based on direction and currency values
    schedule_df['Units'] = schedule_df.apply(lambda row: calculate_contracts(row['FX_Name'], row['Direction'], row['Units']), axis=1)

    # Save the updated DataFrame to a new file
    schedule_df.to_csv(output_file_path, index=False)
output_file_path = 'C:/Users/aksha/Downloads/data_eng_fin/profit and loss/simulator/Trading_Schedule.csv'
# Update the Trading_Schedule.csv file with contract equivalents and save as a new file
save_new_file_with_contracts(file_path, output_file_path)



In [ ]:
import runpy

runpy.run_path('C:/Users/aksha/Downloads/data_eng_fin/profit and loss/simulator/simulator_v2.py')


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 30 14:54:55 2023

@author: aksha
"""

import pandas as pd
import os
import re

# Directory containing the CSV files
folder_path = r'C:\Users\aksha\Downloads\assignment 3'

# Function to process CSV files and calculate the sum
def process_csv_files(directory):
    """
    Process CSV files in the specified directory and calculate the total value for each asset.

    Args:
        directory (str): The directory path where the CSV files are located.

    Returns:
        None
    """
    result_dfs = []
    
    # Traverse the directory to find CSV files
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv") and re.match(r"trade_results_[A-Za-z]+_[0-9]_[0-9]{4}-[0-9]{2}-[0-9]{2}", file):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                df['Total_Value'] = df['Exe_Units'] * df['Exe_Price'] * df['Direction']
                result = df.groupby(['Asset_Name'])['Total_Value'].sum().reset_index()
                result_dfs.append(result)

    # Concatenate all results into a single DataFrame
    final_result = pd.concat(result_dfs, ignore_index=True)

    # Save the final result to a single CSV file
    output_file_path = os.path.join(directory, "combined_result.csv")
    final_result.to_csv(output_file_path, index=False)
    print(f"Combined result saved to {output_file_path}")

# Process CSV files in the specified directory
process_csv_files(folder_path)
